In [3]:
from pathlib import Path
from scapy.all import*
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, UDP
import numpy as np
from scapy.compat import raw


def mask_ip(packet):
    if IP in packet:
        packet[IP].src = "0.0.0.0"
        packet[IP].dst = "0.0.0.0"
    return packet
def remove_ether_header(packet):
    if Ether in packet:
        return packet[Ether].payload
    return packet

def hex_str_to_dec_array(hex_str):
    dec_array = [int(part, 16) for part in hex_str.split(':')]
    return dec_array

def remove_subarray(arr1, arr2):
    arr2_len = len(arr2)
    i = 0
    while i <= len(arr1) - arr2_len:
        if np.array_equal(arr1[i:i + arr2_len], arr2):
            arr1 = np.concatenate((arr1[:i], arr1[i + arr2_len:]))
        else:
            i += 1
    return arr1

mac_arr = ['00:17:88:24:76:ff', '00:1a:22:03:cb:be', '00:1a:22:05:c4:2e', '00:24:e4:24:80:2a', '1c:5f:2b:aa:fd:4e', '20:f8:5e:ca:91:52', '50:c7:bf:00:c7:03', '50:c7:bf:00:fc:a3', '3c:49:37:03:17:db', '3c:49:37:03:17:f0', '5c:cf:7f:06:d9:02', '5c:cf:7f:07:ae:fb', '6c:72:20:c5:17:5a', '74:da:38:23:22:7b', '74:da:38:4a:76:49', '74:da:38:80:79:fc', '74:da:38:80:7a:08', '84:18:26:7b:5f:6b', '90:8d:78:a8:e1:43', '90:8d:78:a9:3d:6f', '90:8d:78:dd:0d:60', '94:10:3e:34:0c:b5', '94:10:3e:35:01:c1', '94:10:3e:41:c2:05', '94:10:3e:42:80:69', '94:10:3e:cd:37:65', 'ac:cf:23:62:3c:6e', 'b0:c5:54:1c:71:85', 'b0:c5:54:25:5b:0e']
input_file = "captures_IoT_Sentinel"
for d in Path(input_file).iterdir():
    device_name = d.name
    count = 0
    for f in d.glob("*.pcap"):
        for pck in PcapReader(str(f)):

            if pck.src in mac_arr:
                mac = pck.src
                pck = remove_ether_header(pck)
                pck = mask_ip(pck)
                arr = np.frombuffer(raw(pck), dtype=np.uint8)
                mac_list = hex_str_to_dec_array(mac)
                arr = remove_subarray(arr, mac_list)
                arr = arr[0:1500]
                if len(arr) < 1500:
                    pad_width = 1500 - len(arr)
                    arr = np.pad(arr, pad_width=(0, pad_width), constant_values=0)
                np.save("features/npy/" + device_name + "/" + device_name + "_" + str(count) + ".npy", arr)
                count += 1
    print(device_name)

Aria
D-LinkCam
D-LinkDayCam
D-LinkDoorSensor
D-LinkHomeHub
D-LinkSensor
D-LinkSiren
D-LinkSwitch
D-LinkWaterSensor
EdimaxCam1
EdimaxCam2
EdimaxPlug1101W
EdimaxPlug2101W
EdnetCam1
EdnetCam2
EdnetGateway
HomeMaticPlug
HueBridge
HueSwitch
iKettle2
Lightify
MAXGateway
README.txt
SmarterCoffee
TP-LinkPlugHS100
TP-LinkPlugHS110
WeMoInsightSwitch
WeMoInsightSwitch2
WeMoLink
WeMoSwitch
WeMoSwitch2
Withings
